In [74]:
import hashlib
import hmac
import json
import os
import time
from datetime import datetime

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [77]:
api_key = os.getenv("BYBIT_API_KEY")
secret_key = os.getenv("BYBIT_SECRET_KEY")

httpClient = requests.Session()

# maximum allowable time difference (in milliseconds) between the server time and the request time
# this parameter helps ensure that the request is not rejected due to time discrepancies
recv_window = str(10000)
url = "https://api.bybit.com"


# Please make sure that the timestamp parameter adheres to the following rule:
# server_time - recv_window <= timestamp < server_time + 1000

def HTTP_Request(endpoint, method, payload):
    time_stamp = str(int(time.time() * 10 ** 3))
    signature = hmac.new(bytes(secret_key, "utf-8"), f"{time_stamp}{api_key}{recv_window}{payload}".encode("utf-8"),
                         hashlib.sha256).hexdigest()
    response = httpClient.request(method, url + endpoint + "?" + payload, headers={
        'X-BAPI-API-KEY': api_key,
        'X-BAPI-SIGN': signature,
        'X-BAPI-SIGN-TYPE': '2',
        'X-BAPI-TIMESTAMP': time_stamp,
        'X-BAPI-RECV-WINDOW': recv_window,
        'Content-Type': 'application/json'
    })
    return response.text


start_date = int(datetime.strptime("2024-01-01", "%Y-%m-%d").timestamp())
end_date = int(datetime.strptime("2024-12-11", "%Y-%m-%d").timestamp())

response = HTTP_Request(endpoint="/v5/market/kline", method="GET",
                        payload=f"category=spot&symbol=ETHUSDT&interval=D&start={start_date}000&end={end_date}000&limit=1000")

In [78]:
# > list[0]: startTime	string	Start time of the candle (ms)
# > list[1]: openPrice	string	Open price
# > list[2]: highPrice	string	Highest price
# > list[3]: lowPrice	string	Lowest price
# > list[4]: closePrice	string	Close price. Is the last traded price when the candle is not closed
# > list[5]: volume	string	Trade volume. Unit of contract: pieces of contract. Unit of spot: quantity of coins
# > list[6]: turnover	string	Turnover. Unit of figure: quantity of quota coin
df = pd.DataFrame(json.loads(response)["result"]["list"]).set_axis(
    ["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"], axis=1)
# .sort_values(
#     "startTime"))
# Convert startTime from unix format to format like "2024-12-01 00:00:00"
df["startTime"] = pd.to_datetime(df["startTime"].astype(int), unit="ms")


In [79]:
df

,startTime,openPrice,highPrice,lowPrice,closePrice,volume,turnover
0,2024-12-09,4004.57,4006.51,3453.99,3688.43,376387.97159,1430354443.0620638
1,2024-12-08,3996.53,4016.13,3923.5,4004.57,127789.89644,508530694.7191617
2,2024-12-07,3998.96,4024.59,3967.48,3996.53,133079.07231,531700698.2738971
3,2024-12-06,3784.94,4087.75,3777.29,3998.96,392949.63146,1550207989.5769901
4,2024-12-05,3837.56,3956.79,3661.02,3784.94,460102.48948,1776515113.7316238
...,...,...,...,...,...,...,...
340,2024-01-04,2209.69,2294.75,2202.1,2267.1,125673.48806,282872485.1691517
341,2024-01-03,2355.2,2385.12,2079.36,2209.69,288021.20403,647269374.3887061
342,2024-01-02,2351.6,2431.08,2339.21,2355.2,154649.65169,367805218.0027896
343,2024-01-01,2282,2351.8,2265.29,2351.6,72164.87871,166736858.7960529
